### Download and compute data

Launch this next code line, if you didn't download data yet

In [ ]:
!python utils/dataHandler.py
!python utils/datasetHandler.py

In [16]:
from sklearn.model_selection import train_test_split

import pandas as pd
import torch.nn as nn
import torch.optim as optim

import torch
import os

In [17]:
dataset_path = "data/dataset/dataset.csv"
df = pd.read_csv(dataset_path, index_col="Date")

train_df, test_df = train_test_split(df,test_size=0.2, shuffle=False)

In [ ]:
class PPO(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(PPO, self).__init__()
        self.hidden_layer1 = nn.Linear(state_dim,128)
        self.hidden_layer2 = nn.Linear(128,128)
        self.critic = nn.Linear(128,1)
        self.actor = nn.Linear(128,action_dim)

    def forward(self, x):
        x = torch.relu(self.hidden_layer1(x))
        x = torch.relu(self.hidden_layer2(x))
        action_probs = torch.softmax(self.actor(x), dim=-1)
        state_value = self.critic(x)
        return action_probs, state_value
    

def init_weights(m):
  if type(m) == nn.Linear:
    nn.init.xavier_uniform_(m.weight)
    m.bias.data.fill_(0.01)

def initialize_ppo_model(device,state_dim,action_dim):
    """
    Initialize the PPO model, optimizer, and loss function.
    """
    model = PPO(state_dim, action_dim).to(device)
    model.apply(init_weights)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    mse_loss = nn.MSELoss()
    return model, optimizer, mse_loss